# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

In [ ]:
# The dealerships want to know what used cars have the highest resale value and why. The 

In [ ]:
# objective is to identify the factors that drive the prices of used cars. To achieve

In [ ]:
# that, our objective is to collect and analyze data related to used car prices and their

In [ ]:
# attributes, and build a predictive model to identify the key drivers of used car prices.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
# After performing EDA I checked for missing values, outliers, duplicates, consistency,

In [ ]:
# accuracy, completeness. In the process I saw anomalies in pricing and odometer. Decided that

In [ ]:
# adding 'age' column is more beneficial than having 'year'. A lot of missing data can be 

In [ ]:
# filled based on 'VIN' column value decoding.

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [ ]:
# Missing values:

In [ ]:
### a lot of them can be based on decoding values of "VIN" column,
### some missing values can be discovered in a differnt column of the same row

In [ ]:
### droppig rows with NaN in "Year" becuase this is a very important parameter
### dropping rows missing 4+ values, because it's to much missing information (25% or more per row)

In [ ]:
### dropping id column because it doesn't contribute to building a model

### dropping VIN column because values are all unique and don't affect outcomes

### dropping the 'model' column (30,000 values). It will just slow down processing time and
### won't have valuable input.

### drop the "size" column because more than 70% of data is missing. BTW "type" 
### column indicates well the size of the car.

In [ ]:
### splitting data by type column to create new datasets
###### Each new dataset will have values in columns that will be appropriate for the vehicle
###### type, e.g. pickup's condition is most likely way different than convirtable's
###### On each new dataset: 
########### no longer need column 'type' (all values same)
########### fill odometer missing values with mean/ median based on values distribution
########### since there is a large pool of data available in each column
################# the mssing values can be replaced with existing values proportionately
########### Ordinal encoding will be applied for 'condition' and 'title_status' 
########### OHE will be applied for other columns

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
# I will build a baseline model to make sure that my models are doing well.

In [ ]:
# For Model training I will use no regularization (optimize polynomial feature degree), L1 
# and L2 (optimize polynomial feature degree and alpha) Linear regression. Then I will 
# run all off them through SFS and RFE (optimize for n features) to hopefully get even 
# better results. 

In [ ]:
# Once I have a best model I will plug it into Permutation Feature Importance to select
# best features.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [1]:
# Once finished with all datasets I will identify the factors that drive the prices of used 
# cars for each type of the vehicle. 
# The earlier phases do not need revisitation nor adjustment. 
# I have information of value to bring back to my client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

In [1]:
# DATA CLEAN UP

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np
import plotly.express as px
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df= pd.read_csv('/Users/chameleonjj/Downloads/ASSIGNMENT 11/Data/vehicles.csv')

In [4]:
df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [6]:
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [7]:
# DATA CLEAN UP

In [8]:
# Checking for missing values in entire DataFrame
missing_values_count = df.isna().sum()
print(missing_values_count)

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
state                0
dtype: int64


In [9]:
## There are some anomalies in a pricing column
### defining the list of values to be removed
to_remove = [1111111, 1111111111, 123456789, 1234567890, 12345678, 1234567, 123456, 12345, 777777, 3736928711, 3024942282, 1000000, 135008900, 10004000]
### using boolean indexing to select the rows that do not contain the values to be removed
df = df[~df['price'].isin(to_remove)]


In [10]:
## There are some anomalies in a odometer column
### It is safe to assume that majority of used cars have mechanical odometers that can record maximum milage of 999,999
### I will replace the milage of > 999999 with NaN value
df.loc[df['odometer'] > 999999, 'odometer'] = np.nan
### It is safe to assume that used cars will have a milage greater than 100 miles
df.loc[df['odometer'] <100, 'odometer'] = np.nan

In [11]:
# Checking changes in missing values
missing_values_count = df.isna().sum()
print(missing_values_count)

id                   0
region               0
price                0
year              1205
manufacturer     17620
model             5250
condition       174043
cylinders       177609
fuel              3013
odometer         11669
title_status      8242
transmission      2556
VIN             160940
drive           130502
size            306275
type             92776
paint_color     130132
state                0
dtype: int64


In [12]:
## Mistakes happen with filling out information and the information can be found in a different column
## I will go through rows to find missing info

In [13]:
### for NaN in 'manufacturer' column
### list of model keywords
model_keywords = ['Isuzu', 'Volkswagen', 'Toyota', 'Mitsubishi', 'Jeep', 'Chrysler', 'Subaru', 'Honda', 'Mazda', 'Nissan', 'Suzuki', 'Lexus', 'Scion', 'Daewoo', 'Kia', 'Hyundai', 'SsangYong', 'Renault', 'VAZ', 'Leyland', 'Mahindra', 'Piaggio', 'Mercedes', 'Ford', 'Proton', 'Jaguar', 'Rover', 'Rolls', 'Bentley', 'DeLorean', 'Lotus', 'Alexander', 'Tata', 'Chevrolet', 'Audi', 'Peugeot', 'Citroen', 'Matra', 'SOE', 'Tauro', 'BMW', 'Mini', 'Volkswagen', 'Opel', 'Maserati', 'Alfa', 'Cagiva', 'Chevy', 'Chev', 'VW', 'Chevy', 'Chevrolet', 'chev', 'vw', 'mercedes', 'rolls']

### applying replacement operation to all rows in 'df'
df['manufacturer'] = df.apply(lambda row: row['manufacturer'] if not pd.isna(row['manufacturer']) else next((kw for kw in model_keywords if kw.lower() in ' '.join([str(val).lower() for val in row.values]).lower()), np.nan), axis=1)


In [14]:
### for NaN in 'type' column
### list of model keywords
model_keywords = ['pickup', 'pickup', 'pickup', 'pickup', 'pickup', 'mini-pickup', 'pickup', 'offroad', 'pickup', 'pickup', 'pickup', 'pickup']

### applying replacement operation to all rows in 'df'
df['type'] = df.apply(lambda row: row['type'] if not pd.isna(row['type']) else next((kw for kw in model_keywords if kw.lower() in ' '.join([str(val).lower() for val in row.values]).lower()), np.nan), axis=1)


In [15]:
### for NaN in 'paint_color' column
### list of model keywords
model_keywords = ['white', 'black', 'red', 'silver', 'gold', 'blue', 'yellow', 'gray', 'brown', 'green', 'orange', 'purple']

### applying replacement operation to all rows in 'df'
df['paint_color'] = df.apply(lambda row: row['paint_color'] if not pd.isna(row['paint_color']) else next((kw for kw in model_keywords if kw.lower() in ' '.join([str(val).lower() for val in row.values]).lower()), np.nan), axis=1)


In [16]:
### for NaN in 'drive' column
### list of model keywords
model_keywords = ['rwd', '4wd', 'fwd']

### applying replacement operation to all rows in 'df'
df['drive'] = df.apply(lambda row: row['drive'] if not pd.isna(row['drive']) else next((kw for kw in model_keywords if kw.lower() in ' '.join([str(val).lower() for val in row.values]).lower()), np.nan), axis=1)


In [17]:
### for NaN in 'fuel' column
### list of model keywords
model_keywords = ['gas', 'diesel', 'hybrid', 'electric']

### applying replacement operation to all rows in 'df'
df['fuel'] = df.apply(lambda row: row['fuel'] if not pd.isna(row['fuel']) else next((kw for kw in model_keywords if kw.lower() in ' '.join([str(val).lower() for val in row.values]).lower()), np.nan), axis=1)


In [18]:
# Checking improvement in missing values in entire DataFrame
missing_values_count = df.isna().sum()
print(missing_values_count)

id                   0
region               0
price                0
year              1205
manufacturer     13018
model             5250
condition       174043
cylinders       177609
fuel              2980
odometer         11669
title_status      8242
transmission      2556
VIN             160940
drive           129834
size            306275
type             92539
paint_color     120436
state                0
dtype: int64


In [19]:
## Based on "VIN' numbers we can decode quite a few Nan fields.
## I will go through standard VIN codes and most popular VIN codes to fill as many NaN fields as possible

In [20]:
### for NaN in 'manufacturer' column
### Defining a dictionary of manufacturer codes and their corresponding names
manufacturer_codes = {
    'AAM': 'Isuzu',
    'AAV': 'Volkswagen',
    'AHT': 'Toyota',
    'JA3': 'Mitsubishi',
    'JA4': 'Mitsubishi',
    'JAA': 'Isuzu',
    'JAB': 'Isuzu',
    'JAC': 'Isuzu',
    'JAE': 'Isuzu',
    'JAL': 'Isuzu',
    'JA7': 'Mitsubishi',
    'JAS': 'Isuzu',
    'JBE': 'Jeep',
    'JCB': 'Chrysler',
    'JF1': 'Subaru',
    'JF2': 'Subaru',
    'JF3': 'Subaru',
    'JF4': 'Subaru',
    'JF5': 'Subaru',
    'JHB': 'Honda',
    'JHL': 'Honda',
    'JHM': 'Honda',
    'JHN': 'Honda',
    'JMB': 'Mazda',
    'JMC': 'Mazda',
    'JN1': 'Nissan',
    'JN2': 'Nissan',
    'JN3': 'Nissan',
    'JN4': 'Nissan',
    'JN5': 'Nissan',
    'JN6': 'Nissan',
    'JN8': 'Nissan',
    'JNB': 'Nissan',
    'JS1': 'Suzuki',
    'JS2': 'Suzuki',
    'JS3': 'Suzuki',
    'JT2': 'Toyota',
    'JT3': 'Toyota',
    'JT4': 'Toyota',
    'JT5': 'Toyota',
    'JT6': 'Lexus',
    'JT8': 'Lexus',
    'JTE': 'Toyota',
    'JTH': 'Lexus',
    'JTL': 'Scion',
    'JTN': 'Toyota',
    'JYA': 'Yamaha',
    'KL1': 'Daewoo',
    'KL2': 'Daewoo',
    'KNB': 'Kia',
    'KNC': 'Kia',
    'KND': 'Kia',
    'KNE': 'Kia',
    'KNG': 'Hyundai',
    'KNH': 'Hyundai',
    'KNL': 'Hyundai',
    'KPA': 'SsangYong',
    'KPT': 'SsangYong',
    'L56': 'Renault',
    'L5Y': 'Renault',
    'LAA': 'VAZ (Lada)',
    'LDV': 'Leyland DAF',
    'MA1': 'Mahindra',
    'MAL': 'Hyundai',
    'MHR': 'Honda',
    'MNB': 'Mitsubishi',
    'MM8': 'Mazda',
    'MP1': 'Piaggio',
    'NLE': 'Mercedes-Benz',
    'NM0': 'Ford',
    'NLH': 'Nissan',
    'PL1': 'Proton',
    'PM1': 'Piaggio',
    'SAJ': 'Jaguar',
    'SAL': 'Land Rover',
    'SAS': 'MG Rover',
    'SCA': 'Rolls-Royce',
    'SCB': 'Bentley',
    'SCE': 'DeLorean',
    'SCM': 'Lotus',
    'SFD': 'Alexander Dennis',
    'SHH': 'Honda',
    'SHS': 'Honda',
    'SJN': 'Nissan',
    'SK9': 'Toyota',
    'SLB': 'Daewoo',
    'SMT': 'Mitsubishi',
    'SN1': 'Tata',
    'TCC': 'Chevrolet',
    'TRU': 'Audi',
    'VF1': 'Renault',
    'VF3': 'Peugeot/Citroen',
    'VF7': 'Citroen',
    'VF8': 'Matra',
    'VN1': 'SOE',
    'VSK': 'Nissan',
    'VSS': 'Suzuki',
    'VV9': 'Tauro',
    'WAU': 'Audi',
    'WBA': 'BMW',
    'WBS': 'BMW M',
    'WBX': 'BMW',
    'WDC': 'Mercedes-Benz',
    'WDD': 'Mercedes-Benz',
    'WMW': 'Mini',
    'WVW': 'Volkswagen',
    'WV1': 'Volkswagen',
    'W0L': 'Opel/Vauxhall',
    'ZAM': 'Maserati',
    'ZAP': 'Piaggio',
    'ZAR': 'Alfa Romeo',
    'ZCG': 'Cagiva'
}

### Looping through the manufacturer codes and update the 'manufacturer' column
for code, name in manufacturer_codes.items():
    df.loc[df['VIN'].str.contains(code) & df['manufacturer'].isna(), 'manufacturer'] = name


In [21]:
# Sometimes the brand has abbreviation. So fixing it
df['manufacturer'] = df['manufacturer'].str.lower()
df['manufacturer'] = df['manufacturer'].replace(['Chevy', 'Chevrolet', 'chev', 'chevy'], 'chevrolet')
df['manufacturer'] = df['manufacturer'].replace(['vw'], 'volkswagen')
df['manufacturer'] = df['manufacturer'].replace(['mercedes'], 'mercedes-benz')
df['manufacturer'] = df['manufacturer'].replace(['bmw m'], 'bmw')
df['manufacturer'] = df['manufacturer'].replace(['rover'], 'land rover')
df['manufacturer'] = df['manufacturer'].replace(['rolls'], 'rolls-royce')
df['manufacturer'] = df['manufacturer'].replace(['alfa-romeo'], 'alfa romeo')

In [22]:
### for NaN in 'year' column
### Defining a dictionary of year codes and their corresponding names
year_codes = {
 
    'P': '1993',
    'R': '1994',
    'S': '1995',
    'T': '1996',
    'V': '1997',
    'W': '1998',
    'X': '1999',
    'Y': '2000',
    '1': '2001',
    '2': '2002',
    '3': '2003',
    '4': '2004',
    '5': '2005',
    '6': '2006',
    '7': '2007',
    '8': '2008',
    '9': '2009',
}

### Looping through the year codes and update the 'year' column based on what code is in 10th position
for code, name in year_codes.items():
    df.loc[df['VIN'].str.contains(code) & (df['VIN'].str.len() > 9) & df['year'].isna(), 'year'] = name


In [23]:
### for NaN in 'type' column
### Defining a dictionary of type codes and their corresponding names
type_codes = {
    '1': 'pickup',
    '3': 'pickup',
    '5': 'pickup',
    '6': 'pickup',
    '7': 'pickup',
}

### Looping through the type codes and update the 'type' column based on what code is in 7th position
for code, name in type_codes.items():
    df.loc[df['VIN'].str.contains(code) & (df['VIN'].str.len() > 6) & df['type'].isna(), 'type'] = name
    
    
### most common type codes
df.loc[df['VIN'].str.contains('1HG') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1FA') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JM1') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('4T1') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('3N1') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1G1') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JTE') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1FM') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1GN') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('5J6') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1J4') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JTE') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1FT') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1GC') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('3D') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('5T') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1GT') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('3C') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1F') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WB') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WDB') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WP') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JM') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WAU') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('SHH') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WVW') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JTD') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WF0') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JM') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('2D') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1FB') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('JT') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('5N1') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WDD') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WBA') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WP') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('WAU') & df['type'].isna(), 'type'] = 'pickup'
df.loc[df['VIN'].str.contains('1G1') & df['type'].isna(), 'type'] = 'pickup'

In [24]:
### for NaN in 'cylinder' column
### most common 'cylinder' codes
### checking if VIN contains 'V8' and 'cylinders' is NaN and replace with '8 cylinders'
df.loc[df['VIN'].str.contains('V8') & df['cylinders'].isna(), 'cylinders'] = '8 cylinders'
df.loc[df['VIN'].str.contains('V10') & df['cylinders'].isna(), 'cylinders'] = '10 cylinders'
df.loc[df['VIN'].str.contains('C12') & df['cylinders'].isna(), 'cylinders'] = '12 cylinders'
df.loc[df['VIN'].str.contains('L4') & df['cylinders'].isna(), 'cylinders'] = '4 cylinders'
df.loc[df['VIN'].str.contains('L6') & df['cylinders'].isna(), 'cylinders'] = '6 cylinders'
df.loc[df['VIN'].str.contains('V6') & df['cylinders'].isna(), 'cylinders'] = '6 cylinders'
df.loc[df['VIN'].str.contains('L4') & df['cylinders'].isna(), 'cylinders'] = '4 cylinders'
df.loc[df['VIN'].str.contains('L3') & df['cylinders'].isna(), 'cylinders'] = '3 cylinders'
df.loc[df['VIN'].str.contains('K3') & df['cylinders'].isna(), 'cylinders'] = '3 cylinders'
df.loc[df['VIN'].str.contains('H3') & df['cylinders'].isna(), 'cylinders'] = '3 cylinders'
df.loc[df['VIN'].str.contains('L5') & df['cylinders'].isna(), 'cylinders'] = '5 cylinders'
df.loc[df['VIN'].str.contains('B5') & df['cylinders'].isna(), 'cylinders'] = '5 cylinders'
df.loc[df['VIN'].str.contains('T5') & df['cylinders'].isna(), 'cylinders'] = '5 cylinders'

### Looping through DataFrame rows
for index, row in df.iterrows():
    vin = row['VIN']
    cylinders = row['cylinders']
    
    # Checking if cylinders value is NaN and VIN contains '6' or 'R' or '3' in its 8th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 7 and vin[7] in ['6', 'R', '3']:
        df.at[index, 'cylinders'] = '6 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains 'L', 'W', 'X', 'N' or 'G' in its 5th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 4 and vin[4] in ['L', 'W', 'X', 'N', 'G']:
        df.at[index, 'cylinders'] = '6 cylinders'
    
    # Checking if cylinders value is NaN and VIN contains 'L', 'W', 'N'in its 6th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 5 and vin[5] in ['L', 'W', 'N']:
        df.at[index, 'cylinders'] = '6 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains 'L', 'W', 'N'in its 7th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 6 and vin[6] in ['L', 'W', 'N']:
        df.at[index, 'cylinders'] = '6 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains 'W','V', 'R','K','B', C', 'E', 'X', 'Y', 'S', 'T' in its 4th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 3 and vin[3] in ['W', 'V','R', 'K','B', 'C', 'E', 'X', 'Y', 'S', 'T']:
        df.at[index, 'cylinders'] = '4 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains 'L','V', 'W','Y','P' in its 8th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 7 and vin[7] in ['L','V', 'W','Y','P']:
        df.at[index, 'cylinders'] = '8 cylinders'    
        
    # Checking if cylinders value is NaN and VIN contains '8' in its 5th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 4 and vin[4] in ['8']:
        df.at[index, 'cylinders'] = '8 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains 'D', 'E', 'F' in its 8th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 7 and vin[7] in ['D', 'E', 'F']:
        df.at[index, 'cylinders'] = 'other'
        
    # Checking if cylinders value is NaN and VIN contains 'D', 'H' in its 7th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 6 and vin[6] in ['D', 'H']:
        df.at[index, 'cylinders'] = 'other'
        
    # Checking if cylinders value is NaN and VIN contains 'T' in its 5th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 4 and vin[4] in ['T']:
        df.at[index, 'cylinders'] = 'other'
        
    # Checking if cylinders value is NaN and VIN contains 'Y' in its 6th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 5 and vin[5] in ['Y']:
        df.at[index, 'cylinders'] = '12 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains '0' in its 7th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 6 and vin[6] in ['0']:
        df.at[index, 'cylinders'] = '12 cylinders'
        
    # Checking if cylinders value is NaN and VIN contains '2' in its 8th position
    if pd.isna(cylinders) and isinstance(vin, str) and len(vin) > 7 and vin[7] in ['2']:
        df.at[index, 'cylinders'] = '12 cylinders'
        

In [25]:
# Checking for missing values in entire DataFrame to figure out the improvement
missing_values_count = df.isna().sum()
print(missing_values_count)

id                   0
region               0
price                0
year                79
manufacturer     12641
model             5250
condition       174043
cylinders        85505
fuel              2980
odometer         11669
title_status      8242
transmission      2556
VIN             160940
drive           129834
size            306275
type             64982
paint_color     120436
state                0
dtype: int64


In [26]:
## DROPPING COLUMNS
## Understanding the columns and its values (I am posting only the results for columns that I will manipulate)

In [27]:
### I can drop id column because it doesn't contribute to building a model
df = df.drop('id', axis=1)

### I can drop VIN column because they are all unique values and don't affect outcomes
df = df.drop('VIN', axis=1)

### I decided to drop the "size" column because more than 70% of data is missing. I also believe that "type" 
### column indicates well the size of the car.
df = df.drop('size', axis=1)

### It looks like there are almost 30,000 values in a 'model' column. It will just slow down processing time and won't have valuable input
df = df.drop('model', axis=1)

In [28]:
## DROPPING ROWS

In [29]:
### Droppig rows that have NaN in column "Year" becuase this is a very important parameter when buying a car.
### Anyway there are only 79 rows in that category

#### creating a new dataframe with only the rows that have NaN in 'year' column so I could expect that data and if anything can be salvaged
df_missing_year = df[df['year'].isnull()].copy()

##### dropping the rows with NaN values in 'year' column from the original dataframe
df.dropna(subset=['year'], inplace=True)



In [30]:
### An age column is better than year column. Hence I will replace it.
# Converting 'year' column from object to float
df['year'] = df['year'].astype(float)

# Subtracting 'year' from current year to create 'age' column
df['age'] = 2022 - df['year']

# Dropping the 'year' column
df = df.drop(columns=['year'])

In [31]:
###  Dropping rows missing 4 values or more, becuase it's to much missing information (25% or more per row)

#### select the specified columns and count the number of NaN values per row
null_counts = df[['fuel', 'odometer', 'title_status', 'transmission', 'manufacturer', 'condition', 'cylinders', 'drive', 'type', 'paint_color']].isna().sum(axis=1)

#### selecting rows with 4 NaN values per row
rows_to_drop = df[null_counts == 4].index

#### dropping the selected rows from the original dataframe 'df'
df.drop(rows_to_drop, inplace=True)

In [32]:
### Droppig rows that have NaN in column "type" becuase this is a very important parameter when buying a car.
### As long as we don't loose more than 20% of data we should be fine. 

#### creating a new dataframe with only the rows that have NaN in 'year' column so I could expect that data and if anything can be salvaged
df_missing_type = df[df['type'].isnull()].copy()

##### dropping the rows with NaN values in 'year' column from the original dataframe
df.dropna(subset=['type'], inplace=True)

In [33]:
# Checking for missing values in entire DataFrame to figure out the improvement
missing_values_count = df.isna().sum()
print(missing_values_count)

region               0
price                0
manufacturer      7491
condition       128699
cylinders        35495
fuel              2398
odometer          7670
title_status      7422
transmission      2072
drive            77283
type                 0
paint_color      68655
state                0
age                  0
dtype: int64


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356000 entries, 27 to 426879
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        356000 non-null  object 
 1   price         356000 non-null  int64  
 2   manufacturer  348509 non-null  object 
 3   condition     227301 non-null  object 
 4   cylinders     320505 non-null  object 
 5   fuel          353602 non-null  object 
 6   odometer      348330 non-null  float64
 7   title_status  348578 non-null  object 
 8   transmission  353928 non-null  object 
 9   drive         278717 non-null  object 
 10  type          356000 non-null  object 
 11  paint_color   287345 non-null  object 
 12  state         356000 non-null  object 
 13  age           356000 non-null  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 40.7+ MB


In [35]:
###COLUMN: 'MANUFACTURER'
### The columns 'manufacturer' has less than 2.2 % of missing data so I decided to 
### replacing NaN with NA manufacturer value
df['manufacturer'] = df['manufacturer'].fillna('NA manufacturer')

In [36]:
# Checking for missing values in entire DataFrame to figure out the improvement
missing_values_count = df.isna().sum()
print(missing_values_count)

region               0
price                0
manufacturer         0
condition       128699
cylinders        35495
fuel              2398
odometer          7670
title_status      7422
transmission      2072
drive            77283
type                 0
paint_color      68655
state                0
age                  0
dtype: int64


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356000 entries, 27 to 426879
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        356000 non-null  object 
 1   price         356000 non-null  int64  
 2   manufacturer  356000 non-null  object 
 3   condition     227301 non-null  object 
 4   cylinders     320505 non-null  object 
 5   fuel          353602 non-null  object 
 6   odometer      348330 non-null  float64
 7   title_status  348578 non-null  object 
 8   transmission  353928 non-null  object 
 9   drive         278717 non-null  object 
 10  type          356000 non-null  object 
 11  paint_color   287345 non-null  object 
 12  state         356000 non-null  object 
 13  age           356000 non-null  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 40.7+ MB


In [38]:
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [39]:
# Summary of data cleaning so far:

# Original data had 426,880 entries and now it has 356000 entries. 
# It is less than 17% loss of data.
# I believe that it is acceptable to loose up to 20% of data and we are withing that range.

# I will not delete anymore data, but manipulate it with median, mode or equivalent percentage fill.

In [40]:
# The remaining data cleaning will be performed on separate dataset to reflect characteristics of each car group.
# I will separate pickup cars out of main data frame. This include any car 25 years old or older.
# I will split remaining dataframe by type. Each type has specific requirements and different usage. For example: 
# a family will be shopping for a different car than a farmer or someone that wants to have fun, fast car.

In [41]:
#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [42]:
# DATAFRAME FOR ANTIQUE CARS (25+ YEARS OLD)

In [43]:
### Cars 25 years and older are considered pickup and they should be separated and have their own ML model
### Dropping rows with antique cars and creating seperate dataframe for them

#### converting the 'year' column to integer data type
df['age'] = pd.to_numeric(df['age'], errors='coerce')

#### creating a new dataframe 'df_antiques' containing only rows with year <= 1997
df_antiques = df[df['age'] >= 25].copy()

#### dropping the rows with age>=25 from the original dataframe 'df'
df.drop(df[df['age'] >= 25].index, inplace=True)

df_antiques.info()
df_antiques.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13547 entries, 137 to 426838
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   region        13547 non-null  object 
 1   price         13547 non-null  int64  
 2   manufacturer  13547 non-null  object 
 3   condition     11331 non-null  object 
 4   cylinders     12759 non-null  object 
 5   fuel          13471 non-null  object 
 6   odometer      12925 non-null  float64
 7   title_status  13402 non-null  object 
 8   transmission  13520 non-null  object 
 9   drive         11838 non-null  object 
 10  type          13547 non-null  object 
 11  paint_color   12053 non-null  object 
 12  state         13547 non-null  object 
 13  age           13547 non-null  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 1.6+ MB


,region,price,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state,age
137,auburn,2500,NA manufacturer,good,6 cylinders,gas,47000.0,clean,manual,rwd,pickup,brown,al,56.0
194,birmingham,33990,kia,NaN,6 cylinders,gas,48784.0,clean,automatic,fwd,SUV,black,al,28.0
219,birmingham,2500,bmw,fair,6 cylinders,gas,110661.0,clean,automatic,rwd,sedan,white,al,27.0
228,birmingham,5500,ford,good,8 cylinders,gas,239000.0,clean,automatic,NaN,truck,green,al,44.0
245,birmingham,5500,mercedes-benz,good,4 cylinders,gas,9999.0,clean,automatic,rwd,convertible,yellow,al,48.0


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342453 entries, 27 to 426879
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        342453 non-null  object 
 1   price         342453 non-null  int64  
 2   manufacturer  342453 non-null  object 
 3   condition     215970 non-null  object 
 4   cylinders     307746 non-null  object 
 5   fuel          340131 non-null  object 
 6   odometer      335405 non-null  float64
 7   title_status  335176 non-null  object 
 8   transmission  340408 non-null  object 
 9   drive         266879 non-null  object 
 10  type          342453 non-null  object 
 11  paint_color   275292 non-null  object 
 12  state         342453 non-null  object 
 13  age           342453 non-null  float64
dtypes: float64(2), int64(1), object(11)
memory usage: 39.2+ MB


In [45]:
###### Each type of car has specific target client group with specific set of requirements.
###### For example: a family of 6 buys a different car than someone looking to have
###### a weekend fun car or a farmer...
###### Therefore I will create seperate ML models (by type) to understand what drives the
###### price in each category.

In [46]:
# Dividing the dataframe by type:
df_sedan = df[df['type'] == 'sedan']
df_SUV = df[df['type'] == 'SUV']
df_pickup = df[df['type'] == 'pickup']
df_truck = df[df['type'] == 'truck']
df_coupe = df[df['type'] == 'coupe']
df_hatchback = df[df['type'] == 'hatchback']
df_convertible = df[df['type'] == 'convertible']
df_van = df[df['type'] == 'van']
df_wagon = df[df['type'] == 'wagon']
df_minivan = df[df['type'] == 'mini-van']
df_bus = df[df['type'] == 'bus']
df_offroad = df[df['type'] == 'offroad']

In [47]:
# Downloading data frames for the ML models
import os
downloads_folder = os.path.expanduser("/Users/chameleonjj/Downloads")
df_sedan.to_csv(os.path.join(downloads_folder, "df_sedan.csv"), index=False)
df_SUV.to_csv(os.path.join(downloads_folder, "df_SUV.csv"), index=False)
df_pickup.to_csv(os.path.join(downloads_folder, "df_pickup.csv"), index=False)
df_truck.to_csv(os.path.join(downloads_folder, "df_truck.csv"), index=False)
df_coupe.to_csv(os.path.join(downloads_folder, "df_coupe.csv"), index=False)
df_hatchback.to_csv(os.path.join(downloads_folder, "df_hatchback.csv"), index=False)
df_convertible.to_csv(os.path.join(downloads_folder, "df_convertible.csv"), index=False)
df_van.to_csv(os.path.join(downloads_folder, "df_van.csv"), index=False)
df_wagon.to_csv(os.path.join(downloads_folder, "df_wagon.csv"), index=False)
df_minivan.to_csv(os.path.join(downloads_folder, "df_minivan.csv"), index=False)
df_bus.to_csv(os.path.join(downloads_folder, "df_bus.csv"), index=False)
df_offroad.to_csv(os.path.join(downloads_folder, "df_offroad.csv"), index=False)
df_antiques.to_csv(os.path.join(downloads_folder, "df_antiques.csv"), index=False)